In [1]:
from dotenv import load_dotenv 

# Load environment variables from .env file
load_dotenv("hse.env")

True

In [10]:
pip install feedparser

  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=fdff5ad144677a14554b9e4ca72ef5492fdd56b825b4fdc9b0e14b8e90d14d42
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
import json
import os
import requests
from openai import OpenAI
import xml.etree.ElementTree as ET
from datetime import datetime
from typing import Dict, List, Any
from dataclasses import dataclass
import feedparser


# Конфигурация системы
@dataclass
class Config:
    MODEL_NAME = "mistralai/mistral-7b-instruct:free"
    OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")
    WEATHER_API_KEY = os.getenv("WEATHERMAP_API_KEY")
    BASE_URL=os.environ.get("OPENAI_BASE_URL")

# Базовый класс агента
class Agent:
    def __init__(self, client: OpenAI):
        self.client = client
        self.tools = self._init_tools()
    
    def _init_tools(self) -> List[Dict]:
        return []
    
    def process(self, messages: List[Dict]) -> Dict:
        response = self.client.chat.completions.create(
            model=Config.MODEL_NAME,
            messages=messages,
            tools=self.tools,
            tool_choice="auto",
        )
        return response.choices[0].message

# Агент погоды
class WeatherAgent(Agent):
    def _init_tools(self) -> List[Dict]:
        return [{
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Получить текущую погоду",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", 
                                     "description": "FIND ONLY the CITY or nearest TOWN in user request, translate it on English"}
                    },
                    "required": ["location"]
                }
            }
        }]
    
    def get_weather(self, location: str) -> str:
        try:
            geo_url = "http://api.openweathermap.org/geo/1.0/direct"
            geo_params = {'q': f"{location},RU", 'limit': 1, 'appid': Config.WEATHER_API_KEY}
            geo_data = requests.get(geo_url, params=geo_params).json()
            
            if not geo_data:
                return json.dumps({"error": "Локация не найдена"})
            
            lat, lon = geo_data[0]['lat'], geo_data[0]['lon']
            weather_url = "https://api.openweathermap.org/data/2.5/weather"
            weather_params = {'lat': lat, 'lon': lon, 'appid': Config.WEATHER_API_KEY, 'units': 'metric'}
            weather_data = requests.get(weather_url, params=weather_params).json()
            
            return json.dumps({
                "location": location,
                "temp": weather_data['main']['temp'],
                "feels_like": weather_data['main']['feels_like'],
                "humidity": weather_data['main']['humidity']
            })
        except Exception as e:
            return json.dumps({"error": str(e)})

# Агент курсов валют
class FinanceAgent(Agent):
    def _init_tools(self) -> List[Dict]:
        return [{
            "type": "function",
            "function": {
                "name": "get_currency",
                "description": "Курсы валют ЦБ РФ",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "currency": {"type": "string", "enum": ["USD", "EUR", "CNY"]}
                    },
                    "required": ["currency"]
                }
            }
        }]
    
    def get_currency(self, currency: str) -> str:
        try:
            response = requests.get('https://www.cbr.ru/scripts/XML_daily.asp')
            root = ET.fromstring(response.content)
            
            for valute in root.findall('Valute'):
                if valute.find('CharCode').text == currency:
                    value = float(valute.find('Value').text.replace(',', '.'))
                    return json.dumps({
                        "currency": currency,
                        "rate": value,
                        "date": root.attrib['Date']
                    })
            return json.dumps({"error": "Валюта не найдена"})
        except Exception as e:
            return json.dumps({"error": str(e)})

# Новый агент новостей
class NewsAgent(Agent):
    def _init_tools(self) -> List[Dict]:
        return [{
            "type": "function",
            "function": {
                "name": "get_news",
                "description": "Новости с сайта ЦБ РФ",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "count": {"type": "integer", "description": "Количество новостей"}
                    },
                    "required": ["count"]
                }
            }
        }]
    


    def get_news(self, count: int = 3) -> str:
        try:
            # Загружаем и парсим RSS-ленту
            feed = feedparser.parse('https://www.cbr.ru/rss/RssNews')
            # Извлекаем до `count` новостей
            news_items = [
                {
                    "title": entry.title,
                    "date": datetime.fromtimestamp(time.mktime(entry.published_parsed)).strftime("%Y-%m-%d")
                    if hasattr(entry, 'published_parsed') else "Unknown"
                }
                for entry in feed.entries[:count]
            ]
            # Возвращаем новости в формате JSON
            return json.dumps({"news": news_items})
        except Exception as e:
            # В случае ошибки возвращаем сообщение об ошибке
            return json.dumps({"error": str(e)})


# Координатор системы
class Orchestrator:
    def __init__(self):
        self.client = OpenAI(
            base_url=Config.BASE_URL,
            api_key=Config.OPENROUTER_API_KEY,
            default_headers={
                "HTTP-Referer": "HSE",
                "X-Title": "Multi-Agent System",
            }
        )
        self.agents = {
            "weather": WeatherAgent(self.client),
            "finance": FinanceAgent(self.client),
            "news": NewsAgent(self.client)
        }
    
    def process_query(self, query: str) -> str:
        # Определяем какие агенты нужны
        needed_agents = self._detect_agents(query)
        
        if not needed_agents:
            return self._simple_response(query)
        
        # Обрабатываем запрос через агентов
        messages = [{"role": "user", "content": query}]
        results = {}
        
        for agent_name in needed_agents:
            agent = self.agents[agent_name]
            agent_response = agent.process(messages)
            
            if agent_response.tool_calls:
                for tool_call in agent_response.tool_calls:
                    func_name = tool_call.function.name
                    args = json.loads(tool_call.function.arguments)
                    
                    if func_name == "get_weather":
                        results["weather"] = agent.get_weather(args["location"])
                    elif func_name == "get_currency":
                        results["finance"] = agent.get_currency(args["currency"])
                    elif func_name == "get_news":
                        results["news"] = agent.get_news(args.get("count", 3))
        
        # Формируем итоговый ответ
        return self._generate_final_response(query, results)
    
    def _detect_agents(self, query: str) -> List[str]:
        prompt = """Анализируй запрос и определи какие модули нужны:
        - weather: запросы о погоде
        - finance: курсы валют
        - news: новости и события в экономике с сайта ЦБ
        Возвращай только список через запятую, например: weather,finance"""
        
        response = self.client.chat.completions.create(
            model=Config.MODEL_NAME,
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": query}
            ],
            temperature=0
        )
        
        agents = response.choices[0].message.content.lower().split(',')
        return [a.strip() for a in agents if a.strip() in self.agents]
    
    def _simple_response(self, query: str) -> str:
        response = self.client.chat.completions.create(
            model=Config.MODEL_NAME,
            messages=[{"role": "user", "content": query}],
            stream=True
        )
        return self._stream_output(response)
    
    def _generate_final_response(self, query: str, data: Dict) -> str:
        context = {
            "query": query,
            "data": data
        }
        
        prompt = f"""Собери ответ на основе данных:
        Запрос: {context['query']}
        Данные: {json.dumps(context['data'], ensure_ascii=False)}
        Ответь развернуто на русском языке"""
        
        response = self.client.chat.completions.create(
            model=Config.MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
            stream=True
        )
        return self._stream_output(response)
    
    def _stream_output(self, response) -> str:
        full_response = ""
        for chunk in response:
            content = chunk.choices[0].delta.content or ""
            print(content, end='', flush=True)
            full_response += content
        return full_response

# Пример использования
if __name__ == "__main__":
    system = Orchestrator()
    
    queries = [
        "Какая погода в Питере и курс Юаня?",
        "Новости ЦБ и курс евро",
        "Расскажи о последних событиях c сайта ЦБ",
    ]
    
    for query in queries:
        print(f"\nЗапрос: {query}")
        print("Ответ:")
        system.process_query(query)
        print("\n" + "="*50)


Запрос: Какая погода в Питере и курс Юаня?
Ответ:
В Санкт-Петербурге на текущий момент температура составляет 10 градусов Цельсия, при этом ощущается температура 7.59 градусов Цельсия. Влажность воздуха составляет 68%. Валютный курс юаня (CNY) к российскому рублю на 22 апреля 2025 года составляет 11.0648 рублей за 1 юань.

Запрос: Новости ЦБ и курс евро
Ответ:
На основе предоставленных данных можно составить следующий ответ:

Новости Центрального Банка России и курса евро на 22 апреля 2025 года:

1. **Новость 1**: "Регуляторные меры Банка России в условиях высокого уровня инфляции". Эта новость подчеркивает текущие действия регулятора в условиях высокой инфляции.

2. **Новость 2**: "Второй квартал 2025 года: экономический анализ и прогнозы". Вторая новость описывает экономическую ситуацию и прогнозы на второй квартал текущего года.

3. **Новость 3**: "Аналитическая записка о текущей ситуации в экономике". Третья новость представляет собой анализ текущей экономической ситуации.

4. **Н